Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# Integrating Non-MONAI Code Into a Bundle

This notebook will discuss strategies for integrating non-MONAI deep learning code into a bundle. This allows existing Pytorch workflows to be integrated into the bundle ecosystem, for example as a distributable bundle for the model zoo or some other repository like Hugging Face, or to integrate with MONAI Label. The assumption taken here is that you already have the components for preprocessing, inference, validation, and other parts of a workflow, and so the task is how to integrate these parts into MONAI types which can be embedded in config files.

In the following cells we'll construct a bundle which follows the [CIFAR10 tutorial](https://github.com/pytorch/tutorials/blob/32d834139b8627eeacb5fb2862be9f095fcb0b52/beginner_source/blitz/cifar10_tutorial.py) in Pytorch's tutorials repo. A number of code components will be copied into the `scripts` directory of the bundle and linked into config files suitable to be used on the command line.

We'll start with an initialised bundle and provide an appropriate metadata file describing the CIFAR10 classification network we'll provide:

In [1]:
%%bash

python -m monai.bundle init_bundle IntegrationBundle
rm IntegrationBundle/configs/inference.json
tree IntegrationBundle

IntegrationBundle
├── configs
│   └── metadata.json
├── docs
│   └── README.md
├── LICENSE
└── models

3 directories, 3 files


In [15]:
%%writefile IntegrationBundle/configs/metadata.json

{
    "version": "0.0.1",
    "changelog": {
        "0.0.1": "Initial version"
    },
    "monai_version": "1.2.0",
    "pytorch_version": "2.0.0",
    "numpy_version": "1.23.5",
    "optional_packages_version": {},
    "name": "IntegrationBundle",
    "task": "Example Bundle",
    "description": "This illustrates integrating non-MONAI code (CIFAR10 classification) into a bundle",
    "authors": "Your Name Here",
    "copyright": "Copyright (c) Your Name Here",
    "data_source": "CIFAR10",
    "data_type": "float32",
    "intended_use": "This is suitable for demonstration only",
    "network_data_format": {
        "inputs": {
            "image": {
                "type": "image",
                "format": "magnitude",
                "modality": "natural",
                "num_channels": 3,
                "spatial_shape": [32, 32],
                "dtype": "float32",
                "value_range": [-1, 1],
                "is_patch_data": false,
                "channel_def": {
                    "0": "red",
                    "1": "green",
                    "2": "blue"
                }
            }
        },
        "outputs": {
            "pred": {
                "type": "probabilities",
                "format": "classes",
                "num_channels": 10,
                "spatial_shape": [10],
                "dtype": "float32",
                "value_range": [0, 1],
                "is_patch_data": false,
                "channel_def": {
                    "0": "plane",
                    "1": "car",
                    "2": "bird",
                    "3": "cat",
                    "4": "deer",
                    "5": "dog",
                    "6": "frog",
                    "7": "horse",
                    "8": "ship",
                    "9": "truck"
                }
            }
        }
    }
}

Overwriting IntegrationBundle/configs/metadata.json
